<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Methodology </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Business Logic with Scopes  </span> </p>


This notebook contains the logic used to perform calculations upon import of data (actuals and cash flows). This is also called 'Data Loading' and the concept of Scope is used here to define the logic and provide the means of executing the logic. Scopes are used to define and perform data handling in a structured and easy-to-read-through fashion.

<br><br>
Import scopes are divided in several notebooks:
- Calculation of Identities
- [Calculation of Present Values](../Import/2ImportScope-PresentValue)
- [Calculation of Actuals](../Import/3ImportScope-Actuals)
- [Calculation of Technical Margin](../Import/4ImportScope-TechnicalMargin)
- [Creation of Ifrs Variables](../Import/5ImportScope-ToIfrsVar)
- [Calculation of Ifrs Variables](../Import/6ImportScope-Compute)

<br><br>
In this notebook the focus is on the defintion of the Identities and on the AoC Step structure and relations required for the computations. 


# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "ImportStorage"

# IModel 

The <code>IModel</code> interface below will be used to execute calculations (i.e. evaluate the scopes) based on imported data.

In [0]:
public interface IModel : IMutableScopeWithStorage<ImportStorage>{}

# Generate Identities

An [Identity](../DataModel/DataStructure#import-identity) is a set of identifiers for a certain set of data. In particular, an identity consists of a certain [AoC Type](../DataModel/DataStructure#aoc-type), [Novelty](../DataModel/DataStructure#novelty), [Data Node](../DataModel/DataStructure#data-node), Accident Year, and information on whether the data is for reinsurance or not. 

Given a certain Data Node and Accident Year, the interface <code>GetIdentities</code> returns all the existing identities (e.g. for Actuals and Cash flows) which have that Data Node and Accident Year.

In [0]:
public interface GetParsedAocSteps : IScope<string, ImportStorage>
{
    IEnumerable<AocStep> Values => GetStorage().GetRawVariables(Identity).Select(x => new AocStep(x.AocType, x.Novelty)).Distinct();
}

In [0]:
public interface GetIdentities : IScope<string, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<GetIdentities>(s => s.WithApplicability<GetCashflowIdentities>(x => x.GetStorage().ImportFormat == ImportFormats.Cashflow));
    
    protected IEnumerable<ImportIdentity> allIdentities => GetStorage().AocConfigurationByAocStep.Values.Select(x => new ImportIdentity {AocType = x.AocType, Novelty = x.Novelty, DataNode = Identity });

    IEnumerable<ImportIdentity> Identities => allIdentities.Select(id => id with { IsReinsurance = GetStorage().DataNodeDataBySystemName[id.DataNode].IsReinsurance,
                                                                                   ValuationApproach = GetStorage().DataNodeDataBySystemName[id.DataNode].ValuationApproach,
                                                                                   LiabilityType = GetStorage().DataNodeDataBySystemName[id.DataNode].LiabilityType });
    IEnumerable<AocStep> AocSteps => Identities.Select(id => new AocStep(id.AocType, id.Novelty)).Distinct();
}

public interface GetCashflowIdentities : GetIdentities
{
    private bool isReinsurance => GetStorage().DataNodeDataBySystemName[Identity].IsReinsurance;
    private IEnumerable<ImportIdentity> ParsedIdentities => GetScope<GetParsedAocSteps>(Identity).Values.Select(aocStep => new ImportIdentity {AocType = aocStep.AocType, Novelty = aocStep.Novelty, DataNode = Identity});
    private IEnumerable<string> rawVariableNovelties => GetStorage().GetRawVariables(Identity).Select(rv => rv.Novelty).Concat(Novelties.C.RepeatOnce()).ToHashSet();
    private IEnumerable<AocStep> calculatedAocSteps => GetStorage().AocConfigurationByAocStep.Values.Where(x => ComputationHelper.CurrentPeriodCalculatedDataTypes.Any(y => x.DataType.Contains(y)) &&
        (!isReinsurance ? !ComputationHelper.ReinsuranceAocType.Contains(x.AocType) : true) && rawVariableNovelties.Contains(x.Novelty) 
        || x.DataType.Contains(DataType.CalculatedProjection) ).Select(x => new AocStep(x.AocType, x.Novelty));
    private IEnumerable<ImportIdentity> specialIdentities => calculatedAocSteps.Select(x => new ImportIdentity {AocType = x.AocType, Novelty = x.Novelty, DataNode = Identity })
        .Concat(GetStorage().AocConfigurationByAocStep.Values.Where(x => (!isReinsurance ? !ComputationHelper.ReinsuranceAocType.Contains(x.AocType) : true) && x.DataType.Contains(DataType.Calculated) && x.Novelty == Novelties.I)
        .Select(aocStep => new ImportIdentity{AocType = aocStep.AocType, Novelty = aocStep.Novelty, DataNode = Identity}));

    IEnumerable<ImportIdentity> GetIdentities.allIdentities => ParsedIdentities.Concat(specialIdentities).Distinct();       
}

# Relevant Amount Types

Similarly, given a certain Data Node and Accident Year, the interface <code>ValidAmountType</code> returns all the amount types which are used in imported data with that Data Node and Accident Year.

In [0]:
public interface ValidAmountType : IScope<string, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
                builder.ForScope<ValidAmountType>(s => s.WithApplicability<BeAmountTypesFromIfrsVariables>(x => x.GetStorage().ImportFormat != ImportFormats.Cashflow ||
                                                                                                                x.GetStorage().IsSecondaryScope(x.Identity)));
    
    IEnumerable<string> BeAmountTypes => GetStorage().GetRawVariables(Identity)
        .Where(rv => rv.AmountType != null).Select(x => x.AmountType).Concat(
            GetStorage().DataNodeDataBySystemName[Identity].IsReinsurance 
                ? GetStorage().DataNodeDataBySystemName[Identity].LiabilityType == LiabilityTypes.LIC 
                    ? new []{AmountTypes.CDR} : new []{AmountTypes.CDRI, AmountTypes.CDR} 
                    : Enumerable.Empty<string>()).ToHashSet();
    
    IEnumerable<string> ActualAmountTypes => GetStorage().GetIfrsVariables(Identity)
        .Where(iv => GetStorage().EstimateTypesByImportFormat[ImportFormats.Actual].Contains(iv.EstimateType))
        .Select(x => x.AmountType).ToHashSet();

    IEnumerable<string> AllImportedAmountTypes => BeAmountTypes.Union(ActualAmountTypes);
}
public interface BeAmountTypesFromIfrsVariables : ValidAmountType
{
    IEnumerable<string> ValidAmountType.BeAmountTypes => GetStorage().GetIfrsVariables(Identity)
        .Where(iv => GetStorage().EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(iv.EstimateType) && iv.AmountType != null)
        .Select(x => x.AmountType).ToHashSet();
}

# AoC Step Structure

The AoC Step structure is constructed from the data which is delivered as input. It is assumed that it depends only on the Group of Contract (i.e. it is invariant across Amount types or Accident Year). 

## Previous AoC Steps
The **PreviousAocSteps** scope computes and provides an important piece of information for each [Identity](../DataModel/DataStructure#import-identity), i.e. for a certain [AoC Type](../DataModel/DataStructure#aoc-type) and [Novelty](../DataModel/DataStructure#novelty).

It provides the list of all previous AoC Steps up to the **BOP** step, whereby a Combined novelty will branch into the InForce and New Business (and all other novelties) AoC Types.

PreviousAocSteps is critical when computing aggregated values along the various dimensions (such as for example Line of Business) and 
is used to compute the Parent AoC Steps and the Reference AoC Steps.

The adjacent AoC Steps prior to Identity one is called Parent Aoc Step (PAS). Thus the PreviousAocSteps is the collection of the identity parents and their parents and so on until there is no parent.

$$
\text{PreviousAocSteps}(\rm{AocStep}) = \{PAS_1, PAS_2, \ldots\}
$$
where
$$
\rm{PAS}_1 = \rm{ParentAocStep}(\rm{AoC Step})
$$
$$
\rm{PAS}_2 = \rm{ParentAocStep}(\rm{PAS}_1).
$$

This scope depends on the InputSource (Actual or Cashflow) for which the PreviousSteps are requested due to the Aoc Chain differences between Actual reports and the rest.

The exact structure being return depends on the **order** of the AoC Steps (which is set by the [AoC Step Configuration](../DataModel/DataStructure#aoc-configuration)), and on which AoC steps exist.

In [0]:
public interface PreviousAocSteps : IScope<(ImportIdentity Id, StructureType AocStructure), ImportStorage>
{   
    private AocStep identityAocStep => new AocStep(Identity.Id.AocType, Identity.Id.Novelty);
    private int aocStepOrder => GetStorage().AocConfigurationByAocStep[identityAocStep].Order;
    private IEnumerable<AocStep> aocChainSteps => GetStorage().GetAllAocSteps(Identity.AocStructure);
    IEnumerable<AocStep> Values => aocChainSteps.Contains(identityAocStep)
        ? GetScope<GetIdentities>(Identity.Id.DataNode).AocSteps
            .Where(aoc => aocChainSteps.Contains(aoc) && GetStorage().AocConfigurationByAocStep[aoc].Order < aocStepOrder && 
                          (Identity.Id.Novelty != Novelties.C ? aoc.Novelty == Identity.Id.Novelty : true) )
            .OrderBy(aoc => GetStorage().AocConfigurationByAocStep[aoc].Order)
        : Enumerable.Empty<AocStep>();
} 

## Parent AoC Step 
The **ParentAocStep** scope computes and provides an important piece of information for each [Identity](../DataModel/DataStructure#import-identity), i.e. for a certain [AoC Type](../DataModel/DataStructure#aoc-type) and [Novelty](../DataModel/DataStructure#novelty).

It provides the list of the adjacent AoC Steps prior to Identity one. It can be more than one only for the step **CL** where a parent for each novelty is considered.
ParentAocStep is critical when computing the *telescoping* differences. 

They are defined as follows:

$$
\text{ParentAocStep}(\text{AoC Step}) = \left\{
\begin{array}{cl}
\text{empty} & \text{if AoC Type BOP or is of DataType Calculated} \\
\text{Last of previous AoC Step for each Novelty} & \text{if not BOP or Calculated but it is the first Combined AoC Step} \\
\text{Last of previous AoC Step of the same Novelty} & \text{if not BOP or Calculated and not the first Combined AoC Step} \\
\end{array}
\right.
$$

where the last AoC Step is obtained by ordering the AoC Steps according to their order (as defined by its AoC Type) and taking the last one.
Note that the DataType of an AocStep is defined in the [AoC Step Configuration](../DataModel/DataStructure#aoc-configuration) table.


In [0]:
public interface ParentAocStep : IScope<(ImportIdentity Id, string AmountType, StructureType AocStructure), ImportStorage>
{
    private IEnumerable<AocStep> CalculatedAocStep => GetStorage().AocConfigurationByAocStep.Where(kvp => kvp.Value.DataType.Contains(DataType.Calculated)).Select(kvp => kvp.Key);
    
    private IEnumerable<AocStep> TelescopicStepToBeRemoved => Identity.AmountType == AmountTypes.CDR ? Enumerable.Empty<AocStep>() : GetStorage().AocConfigurationByAocStep.Where(kvp => kvp.Value.AocType == AocTypes.CRU).Select(kvp => kvp.Key);
    private IEnumerable<AocStep> PreviousAocStepsNotCalculated => GetScope<PreviousAocSteps>((Identity.Id, Identity.AocStructure)).Values.Where(aoc => !CalculatedAocStep.Concat(TelescopicStepToBeRemoved).Contains(aoc));
    private bool IsFirstCombinedStep => Identity.Id.Novelty == Novelties.C && !PreviousAocStepsNotCalculated.Any(aoc => aoc.Novelty == Novelties.C);
    private bool IsCalculatedStep => CalculatedAocStep.Contains(new AocStep(Identity.Id.AocType, Identity.Id.Novelty));

    IEnumerable<AocStep> Values => (Identity.Id.AocType == AocTypes.BOP || IsCalculatedStep, IsFirstCombinedStep) switch {
        (true, _ ) => Enumerable.Empty<AocStep>(),
        (false, true) => PreviousAocStepsNotCalculated.GroupBy(g => g.Novelty, (g, val) => val.Last()),
        (false, false) => PreviousAocStepsNotCalculated.Last(aoc => aoc.Novelty == Identity.Id.Novelty).RepeatOnce(),
    };
}

## Reference AoC Step

The **ReferenceAocStep** scope provides the AoC Step from which to retrieve the data in order to compute its value (e.g. AoC Step EA retrieves Present Values, while AoC Step YCU retrieves Nominal).

They are defined as follows:

$$
\text{ReferenceAocStep}(\text{AoC Step}) = \left\{
\begin{array}{cl}
\text{The last AoC Step with Data Type != Calculated} ~, & \text{if AoC Step is of DataType Calculated or CalculatedTelescopic} \\
\text{As defined by a custom setting} & \text{otherwise if provided custom settings} \\
\text{self} & \text{all other cases} \\
\end{array}
\right.
$$

where the last AoC Step is obtained by ordering the AoC Steps according to their order (as defined by its AoC Type) and taking the last one.
Custom settings for the Reference AoC Step is given in [helper variables](../Utils/ImportCalculationMethods#helper-variables-used-in-calculation) and can be overwritten in custom projects. 

In [0]:
// The Reference AocStep from which the data (Nominal or PV) is retrieved to to compute the current AoC Step
public interface ReferenceAocStep : IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
                builder.ForScope<ReferenceAocStep>(s => s.WithApplicability<ReferenceAocStepForProjections>(x => x.GetStorage().GetShift(x.Identity.ProjectionPeriod) >= MonthInAYear));

    protected AocStep identityAocStep => new AocStep(Identity.AocType, Identity.Novelty);

    protected IEnumerable<AocStep> referenceForCalculated => GetScope<PreviousAocSteps>((Identity, StructureType.AocPresentValue)).Values
        .GroupBy(g => g.Novelty, (g, val) => val.Last(aocStep => !ComputationHelper.CurrentPeriodCalculatedDataTypes.Any(dt => GetStorage().AocConfigurationByAocStep[aocStep].DataType.Contains(dt))));
                
    protected bool IsCalculatedAocStep => ComputationHelper.CurrentPeriodCalculatedDataTypes.Any(dt => GetStorage().AocConfigurationByAocStep[identityAocStep].DataType.Contains(dt));
    
    IEnumerable<AocStep> Values => (
        IsCalculatedAocStep, 
        ComputationHelper.ReferenceAocSteps.TryGetValue(identityAocStep, out var CustomDefinedReferenceAocStep) //IsCustomDefined
        ) switch {
            (true, false) => referenceForCalculated.Any(x => x.Novelty == Novelties.C) ? referenceForCalculated.Where(x => x.Novelty == Novelties.C) : referenceForCalculated,
            (true, true) => CustomDefinedReferenceAocStep,
            (false, _) => identityAocStep.RepeatOnce(),
            };
}

public interface ReferenceAocStepForProjections : ReferenceAocStep
{
    private bool IsInforce => Identity.Novelty == Novelties.I;

    IEnumerable<AocStep> ReferenceAocStep.Values => (
        IsCalculatedAocStep, 
        ComputationHelper.ReferenceAocSteps.TryGetValue(identityAocStep, out var CustomDefinedReferenceAocStep), //IsCustomDefined
        IsInforce
        ) switch {
            (true, false, false) => referenceForCalculated.Any(x => x.Novelty == Novelties.C) ? referenceForCalculated.Where(x => x.Novelty == Novelties.C) : referenceForCalculated,
            (true, false, true) or (false, false, true) => new []{new AocStep(AocTypes.CL, Novelties.C)},
            (true, true, _) or (false, true, true) => CustomDefinedReferenceAocStep,
            (false, true, false) or (false, false, false) => identityAocStep.RepeatOnce(),
            };
}